# Simulating coupled Classical Harmonic oscillators using Classiq
___

**Authors**: Viraj Dsouza, Cristina Radian, Kerem Yurtseven

**Date**: 09.08.2024

This work is designed under and for the final project of the Womanium Quantum+AI Program 2024

**Main Reference**: Exponential Quantum Speedup in Simulating Coupled Classical Oscillators, Babbush et.al, 2023 [1](https://arxiv.org/abs/2303.13012)
___

## Theoretical Background:


We are trying to find a solution for a system of classical coupled harmonic oscillators with $N=2^n$ masses. Let $\vec{x}$ be the displacement vector
$$
    \vec{x} = \left(x_1(t),.....,x_N(t) \right)^T \in \mathbb{R}^N
    \qquad (1)
$$
So, for the system of $N$ masses connected by springs, Newton's equation becomes
$$
    m_j \ddot{x}_j(t) = \sum_{k \neq j} \kappa_{jk} \left( x_k(t) - x_j(t) \right) - \kappa_{jj} x_j(t) \qquad (2)
$$
where $j \in [N] := \{1,...,N\}$ and $\kappa_{jk}$ is the spring constant of the spring connecting the $j ^{th}$ and the $k ^{th}$ masses.

Representing Newton's Equation in matrix form:
$$
    \mathbf{M}\ddot{\vec{x}}(t) = - \mathbf{F}\vec{x}(t)
    \qquad (3)
$$
where $\mathbf{M}$ is the $N\times N$ diagonal matrix with entries $m_j > 0$ and $\mathbf{F}$ is an $N\times N$ matrix with $f_{jj}=\sum_k \kappa_{jk}$ and $f_{jk} = -\kappa_{jk}$.

___



In order to reduce Newton's Equation to a Quantum Evolution, we perform a change of variable where $\vec{y}(t) := \sqrt{\mathbf{M}}\vec{x}(t)$. Equation $(3)$ becomes:
$$
    \ddot{\vec{y}}(t) = -\mathbf{A}\vec{y}(t)
    \qquad (4)
$$
where $\mathbf{A} := \sqrt{\mathbf{M}}^{-1}\mathbf{F}\sqrt{\mathbf{M}}^{-1}$. Convert equation (4) into a similar form of Schrödinger's Equation by adding $i\sqrt{\mathbf{A}}\dot{\vec{y}}(t)$ on both sides of equation $4$,
$$
    \ddot{\vec{y}}(t) + i\sqrt{\mathbf{A}}\dot{\vec{y}}(t)=i\sqrt{\mathbf{A}}\left(\dot{\vec{y}}(t) + i\sqrt{\mathbf{A}}\vec{y}(t)\right)
    \qquad (5)
$$
Solution to (5) is
$$
    \dot{\vec{y}}(t) + i\sqrt{\mathbf{A}}\vec{y}(t) = e^{it\sqrt{\mathbf{A}}}\left(\dot{\vec{y}}(0) + i\sqrt{\mathbf{A}}\vec{y}(0)\right)
    \qquad (6)
$$

___

In order to access $\sqrt{\mathbf{A}}$, define $\mathbf{B}\mathbf{B^\dagger} = \mathbf{A}$ where B is $N\times M$ and $M = N\times (N+1) /2$. However, to simulate the Hamiltonian using qubits, we need it to be a square matrix of dimension $2^{q}$ where $q$ is the number of qubits simulating $H$. To achieve this, we  pad $B$ with zeros to make its dimension $N^2\times N^2$, so that $H$ becomes a $2N^{2}X2N^{2}$ matrix which can be simulated with $q=2n+1$ qubits, where $N=2^{n}$. So, the hamiltonian after padding becomes
$$
    \mathbf{H} := -\begin{pmatrix}
\mathbf{0}{_{N^{2}XN^{2}}} & \mathbf{B}{_{N^{2}XN^{2}}}\\
\mathbf{B^\dagger}{_{N^{2}XN^{2}}} & \mathbf{0}{_{N^{2}XN^{2}}}
\end{pmatrix}_{2N^{2}X2N^{2}}
\qquad (7)
$$
(5) can be stated as:
$$
    |{\dot{\psi}(t)}\rangle = -i\mathbf{H} |{\psi(t)}\rangle
    \qquad (8)
$$
where
$$
    |{\psi(t)}\rangle \propto \begin{pmatrix}
\dot{\vec{y}}(t)\\
i\mathbf{B^\dagger}\vec{y}(t)
\end{pmatrix}
\qquad (9)
$$
is an appropriate solution to equation $(8)$

Here $\mathbf{A}$ is positive-semi definite and Hermitian by definition, Cholesky decomposition cannot be used to find $\mathbf{B}$ always since it requires $A$ to be positive definite. A method to find $\mathbf{B}$ is given by the paper
$$
    \sqrt{\mathbf{M}}\mathbf{B}|{j,k}\rangle=
    \begin{cases}
        \sqrt{\kappa_{jj}} & \text{if } j = k\\
        \sqrt{\kappa_{jk}}\left(|{j}\rangle-|{k}\rangle \right) & \text{if } j < k
    \end{cases}
    \qquad (10)
$$
where $|{j,k}\rangle$ is the bra-ket notation for the basis of complex space $\mathbb{C}^{M}$ over $\mathbb{C}$ and $j \leq k \in [N]$.
___

<!-- ##### A brief background on vector spaces:

Here is a crucial point for the difference between basis over $\mathbb{C}$ and basis over $\mathbb{R}$. If basis are given over $\mathbb{C}$, they are the complex combinations of the bases and if bases are given over $\mathbb{R}$, they are the real combinations of the basis vectors. To illustrate, let us take the bases of $\mathbb{C}^2$ and $\gamma \in \mathbb{C}^2$.
$$
    \vec{\gamma} = a_1\begin{pmatrix}
            1\\
            0
            \end{pmatrix} + a_2\begin{pmatrix}
            0\\
            1
            \end{pmatrix} + a_3\begin{pmatrix}
            i\\
            0
            \end{pmatrix} + a_4\begin{pmatrix}
            0\\
            i
            \end{pmatrix}
    \qquad (11)
$$
(11) is the span for $\gamma$ using bases of $\mathbb{C}^2$ over $\mathbb{R}$ and $a_i \in \mathbb{R}$.
$$
    \vec{\gamma} = c_1\begin{pmatrix}
            1\\
            0
            \end{pmatrix} + c_2\begin{pmatrix}
            0\\
            1
            \end{pmatrix}
    \qquad (12)
$$
(12) is the span for  $\gamma$ using bases of $\mathbb{C}^2$ over $\mathbb{C}$ and $c_i \in \mathbb{C}$. -->


As an example, consider two unit masses $m_1$ and $m_2$, also let $k_{11}=k_{22}=0$ and $k_{12}=k_{21}=1$

In order to create $\mathbf{B}$, use (10).

For $k=j=1$:
$$
\sqrt{\mathbf{M}}\mathbf{B}|{1,1}\rangle = \sqrt{\kappa_{11}}|{1}\rangle

$$

$$
\mathbf{B}|{1,1}\rangle = \begin{pmatrix}
    0 \\
    0
\end{pmatrix} 
$$

$$
\mathbf{B}\begin{pmatrix}
    1 \\
    0 \\
    0
\end{pmatrix} = \begin{pmatrix}
    0 \\
    0
\end{pmatrix} \qquad (15)
$$

For $k=1$ and $j=2$:
$$
\sqrt{\mathbf{M}}\mathbf{B}|{1,2}\rangle = \sqrt{\kappa_{12}}\left( |{1}\rangle - |{2} \rangle \right) 
$$

$$
\mathbf{B}|{1,2}\rangle = \left( |{1}\rangle - |{2} \rangle \right) 
$$

$$
\mathbf{B} \begin{pmatrix}
    0 \\
    1 \\
    0
\end{pmatrix} = \begin{pmatrix}
    1 \\
    0
\end{pmatrix} - \begin{pmatrix}
    0 \\
    1
\end{pmatrix} 
$$

$$
\mathbf{B} \begin{pmatrix}
    0 \\
    1 \\
    0
\end{pmatrix} = \begin{pmatrix}
    1 \\
    -1
\end{pmatrix} \qquad (16)
$$

For $k=j=2$:
$$
\sqrt{\mathbf{M}}\mathbf{B}|{2,2}\rangle = \sqrt{\kappa_{22}}|{2} \rangle 
$$

$$
\mathbf{B}|{2,2}\rangle = \begin{pmatrix}
    0 \\
    0
\end{pmatrix} 
$$

$$
\mathbf{B} \begin{pmatrix}
    0 \\
    0 \\
    1
\end{pmatrix} = \begin{pmatrix}
    0 \\
    0
\end{pmatrix} \qquad (17)
$$
Combining $(15), (16), (17)$
$$
\mathbf{B}\begin{pmatrix}
    1 & 0 & 0 \\
    0 & 1 & 0 \\
    0 & 0 & 1
\end{pmatrix} = \begin{pmatrix}
    0 & 1 & 0 \\
    0 & -1 & 0
\end{pmatrix} 
$$

$$
\mathbf{B} = \begin{pmatrix}
    0 & 1 & 0 \\
    0 & -1 & 0
\end{pmatrix} \qquad (18)
$$

Lastly, show that $\mathbf{B}\mathbf{B}^\dagger = \mathbf{A}$
$$
\begin{pmatrix}
    0 & 1 & 0 \\
    0 & -1 & 0
\end{pmatrix} \begin{pmatrix}
    0 & 0 \\
    1 & -1 \\
    0 & 0
\end{pmatrix} = \begin{pmatrix}
    1 & -1 \\
    -1 & 1
\end{pmatrix} \qquad (19)
$$
That ends the constitution of $\mathbf{B}$ for this example.

Following equation $(7)$ the Hamiltonian ($H$) for this case becomes
$$
\mathbf{H} = \begin{pmatrix}
    0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & -1 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    1 & -1 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
    0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{pmatrix} \qquad (20)
$$

___

We can also use the following methodology (for the simplest case) to obtain $B$ from $A$.

For a square and **normal** matrix $A$ we have the eigen value decomposition as follows: 

$$
A = V \Lambda V^T
$$

where:
    $V$ is the matrix of eigenvectors of $A$.
    $\Lambda$ is the diagonal matrix of eigenvalues of $A$.


Next, we construct the diagonal matrix $\sqrt{\Lambda}$ containing the square roots of the eigenvalues:

$$
\sqrt{\Lambda} = \text{diag}(\sqrt{\lambda_1}, \sqrt{\lambda_2}, \ldots, \sqrt{\lambda_n})
$$

We then define the matrix $B$ as follows:

$$B = V \sqrt{\Lambda}$$


Its easy to verify that $A$ can be reconstructed from $B$ as $ A= BB^T$

___


## Simulating the simplest Possible case:

In [22]:
!pip install classiq==0.43.3 

  Using cached classiq-0.43.3-py3-none-any.whl.metadata (3.1 kB)
  Using cached packaging-22.0-py3-none-any.whl.metadata (3.1 kB)
  Using cached sympy-1.10.1-py3-none-any.whl.metadata (12 kB)
Using cached classiq-0.43.3-py3-none-any.whl (387 kB)
Using cached packaging-22.0-py3-none-any.whl (42 kB)
Using cached sympy-1.10.1-py3-none-any.whl (6.4 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
  Attempting uninstall: packaging
    Found existing installation: packaging 23.2
    Uninstalling packaging-23.2:
      Successfully uninstalled packaging-23.2
  Attempting uninstall: classiq
    Found existing installation: classiq 0.44.0
    Uninstalling classiq-0.44.0:
      Successfully uninstalled classiq-0.44.0


In [2]:
## Import Libraries
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import typing
import itertools
import scipy
import classiq
from classiq import *
from classiq.execution import ExecutionPreferences, ClassiqBackendPreferences, ClassiqSimulatorBackendNames, ExecutionSession
from typing import cast
from qutip import qeye, sigmax, sigmay, sigmaz
from itertools import product
from numpy import kron
from scipy.special import eval_chebyt, jv
from classiq.qmod.symbolic import pi

## Defining all Global Variables
PAULI_DICT = {
    "I": np.array([[1, 0], [0, 1]], dtype=np.complex128),
    "Z": np.array([[1, 0], [0, -1]], dtype=np.complex128),
    "X": np.array([[0, 1], [1, 0]], dtype=np.complex128),
    "Y": np.array([[0, -1j], [1j, 0]], dtype=np.complex128),
}

CHAR_TO_STUCT_DICT = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}

### Function Declarations for Pre-processing classical inputs for loading as quantum inputs:

In [3]:
def is_power_of_2(n):
    """Check if a number is a power of 2."""
    return (n & (n - 1) == 0) and n != 0

def initialize_system():
    while True:
        N = int(input("Enter the number of masses (N): "))
        if is_power_of_2(N):
            break
        else:
            print("Error: N must be a power of 2. Please try again.")

    # Initialize the mass matrix (M) and spring constant matrix (K)
    M = np.zeros((N, N))
    K = np.zeros((N, N))

    # Input the values of the masses
    for i in range(N):
        M[i, i] = float(input(f"Enter the mass m_{i+1}: "))

    # Input the spring constants
    for i in range(N):
        for j in range(i, N):
            k = float(input(f"Enter the spring constant k_{i+1}{j+1}: "))
            K[i, j] = k
            K[j, i] = k  # Since K is a symmetric matrix

    # Input the initial position vector (x_0)
    x_0 = np.zeros((N, 1))
    for i in range(N):
        x_0[i, 0] = float(input(f"Enter the initial position x_0_{i+1}: "))

    # Input the initial velocity vector (xdot_0)
    xdot_0 = np.zeros((N, 1))
    for i in range(N):
        xdot_0[i, 0] = float(input(f"Enter the initial velocity xdot_0_{i+1}: "))

    return N, M, K, x_0, xdot_0


def create_matrix_F(K, N):
    """
        Definition:
            Compose the F matrix using the spring coefficients
        Args:
            K (nd_array NxN): Spring Coefficients - Symmetric Matrix
            N (int): Number of masses - should be 2^n
        Outputs:
            F (nd_array NxN): F matrix 
    """

    F = np.zeros((N,N))

    for i in range(N):
        for j in range(N):
            if i == j: # Sum of kappa_{ik} over all k in 1 to N
                tmp = 0
                for l in range(N):
                    tmp += K[i][l]
                F[i][i] = tmp
            if i != j: # -kappa_{ij}
                F[i][j] = -K[i][j]
    
    return F
            
def create_matrix_A(M, F):
    """
        Definition:
            Compose the A matrix using F matrix and mass matrix M
        Args:
            M (nd_array NxN): Mass Matrix - Diagonal Matrix
            F (nd_array NxN): F matrix - Symmetric Matrix
        Outputs:
            A (nd_array NxN): A matrix - PSD and Real Symmetric
    """

    rootM = np.sqrt(M)
    rootM_inverse = np.linalg.inv(rootM)

    A = np.linalg.multi_dot([rootM_inverse, F, rootM_inverse])

    return A

def coordinate_transformation(M, x_0, xdot_0):
    """
        Definition:
            Transform x coordinates to y coordinates
        Args:
            M (nd_array NxN): Mass Matrix - Diagonal Matrix
            x_0 (nd_array 1xN): Initial Positions
            xdot_0 (nd_array 1xN): Initial Velocities
        Outputs:
            Y (dict): Dictionary containing transformed coordinates    
                y_0 (nd_array 1xN): Transformed Positions
                ydot_0 (nd_array 1xN): Transformed Velocities        
    """

    y_0 = np.matmul(np.sqrt(M), x_0)
    ydot_0 = np.matmul(np.sqrt(M), xdot_0)

    Y = {"y_0": y_0, "ydot_0": ydot_0}

    return Y

def create_matrix_B(M, K, A, N):
    """
        Definition:
            Compose B matrix using eq.16
        Args:
            M (nd_array NxN): Mass Matrix - Diagonal Matrix
            K (nd_array NxN): Spring Coefficients - Symmetric Matrix
            A (nd_array NxN): A matrix - PSD and Real Symmetric
            N (int): Number of masses - should be 2^n
        Outputs:
            B (nd_array NxM): B Matrix - M = N * (N + 1) / 2
    """

    B = np.zeros((N,int(N*(N+1)/2)), dtype=complex)
    
    rootM = np.sqrt(M)
    rootM_inverse = np.linalg.inv(rootM)

    for i in range(N):
        for j in range(i,N):
            tmp = np.zeros((N,1))
            
            if i == j:
                bra = np.zeros((N,1))
                bra[i][0] = 1
                tmp = np.sqrt(K[i][j]) * bra
            
            if i != j:
                bra0 = np.zeros((N,1))
                bra1 = np.zeros((N,1))
                bra0[i][0] = 1
                bra1[j][0] = 1
                tmp = np.sqrt(K[i][j]) * (bra0 - bra1)

            tmpcol = np.matmul(rootM_inverse, tmp)

            for l in range(N):
                B[l][i+j] = tmpcol[l][0]

    return B
    
def padding_B(B, N):
    """
        Definition:
            Pad B with zeros to make it square
        Args:
            B (nd_array NxM): B Matrix - M = N * (N + 1) / 2
            N (int): Number of masses - should be 2^n
        Outputs:
            B_padded (nd_array N^2xN^2): Padded B Matrix
    """
    
    B_padded = np.zeros((int(N**2), int(N**2)), dtype=complex)
    B_padded[:N, :int(N*(N+1)/2)] = B

    return B_padded

def create_Hamiltonian(B):
    """
        Definition:
            Compose Hamiltonian Matrix eq.12
        Args:
            B (nd_array N^2xN^2): Padded B Matrix
        Outputs:
            H (nd_array 2*N^2x2*N^2): Padded Hamiltonian - Hermitian Matrix
    """

    H = -np.block([[np.zeros((B.shape[0], B.shape[0])), B], [B.conj().T, np.zeros((B.shape[1], B.shape[1]))]])

    return H

def create_init_state(B, y_0, ydot_0, N):
    """
        Definition:
            Create the initial state eq.14
        Args:
            B (nd_array N^2xN^2): Padded B Matrix
            y_0 (nd_array 1xN): Transformed Positions
            ydot_0 (nd_array 1xN): Transformed Velocities 
        Outputs:
            init_state (nd_array 1x2*N^2): Initial State to be using Hamiltonian Evolution  
    """

    y_0_padded = np.pad(y_0, ((0, N**2-N), (0, 0)), mode='constant', constant_values=0)
    init_lower = 1j * np.matmul(B.T, y_0_padded)
    ydot_0_padded = np.pad(ydot_0, ((0, N**2-N), (0, 0)), mode='constant', constant_values=0)

    init_state = np.vstack((ydot_0_padded, init_lower.reshape(-1, 1)))

    return init_state

def normalize_init_state(init_state):
    """
        Definition:
            Normalize the initial state
        Args:
            init_state (nd_array 1x2*N^2): Initial State to be using Hamiltonian Evolution  

        Outputs:
            normalization (dict): Dictionary containing normalized initial state and normalization factor
                norm (int): Normalization factor
                normalized_init_state (1D list 2*N^2): Normalized Initial State with Real Amplitudes (add pi/2 phase in evolution)
    """

    amp = 0
    for i, element in enumerate(init_state):
        amp += element * element.conj()

    norm = np.sqrt(amp)
    init_state_normalized_complex = init_state / norm

    half_index = int(len(init_state_normalized_complex) / 2)
    normalized_init_state = init_state_normalized_complex
    normalized_init_state[half_index:, 0] = -1j * init_state_normalized_complex[half_index:, 0]

    return {"norm": norm, "normalized_init_state": normalized_init_state[:,0]}

def sum_amplitudes_by_state(simulated_states):
    """
        Definition:
            Sum all the states that corresponds to same output qnum
        Args:
            simulated_states (list): Contains the state vector simulation results
        Output:
            state_amplitude_sums (dict): Amplitude and phase of each output qnum
    """
    state_amplitude_sums = {state: 0 for state in range(8)}
    
    for sim_state in simulated_states:
        state_value = sim_state.state['state']
        if state_value in state_amplitude_sums:
            state_amplitude_sums[state_value] += sim_state.amplitude
    
    return state_amplitude_sums

def normalize_final_state(state_amplitudes, parsed_state_vector):
    """
        Definition:
            Normalize the final state
        Args:
            state_amplitude_sums (dict): Amplitude and phase of each output qnum
            parsed_state_vector (dict): Results of the state vector simulation
        Output:
            final_state (nd_array 2*N^2x1): Normalized final state taking phases into account
    """

    values = np.array([state_amplitudes[i] for i in range(len(state_amplitudes))])
    global_phase = np.angle(values)[0]
    values = values / np.exp(1j * global_phase)

    normalization = np.linalg.norm(
            [
                parsed_state.amplitude
                for parsed_state in parsed_state_vector
            ]
        )

    final_state = values / normalization

    return final_state

def simplify_final_state(final_state):
    """
        Definition:
            Normalize the final state
        Args:
            final_state (nd_array 2*N^2x1): Normalized final state taking phases into account
        Output:
            simplified_final_state (nd_array 2*N^2x1): Final state with rounding
    """
    
    epsilon = 10**(-4)

    simplified_final_state = final_state
    for i, value in enumerate(final_state):
        imag = np.imag(value)
        real = np.real(value)
        
        if abs(np.real(value)) < epsilon:
            real = 0
        if abs(np.imag(value)) < epsilon:
            imag = 0
        simplified_final_state[i] = real + 1j*imag

    return simplified_final_state

def calculate_energy(pos, vel, M, K):
    """
        Definition:
            Calculate the energy at a specific time
        Args:
            pos (nd_array Nx1): Positions at a specific time
            vel (nd_array Nx1): Velocities at a specific time
            M (nd_array NxN): Mass Matrix - Diagonal Matrix
            K (nd_array NxN): Spring Coefficients - Symmetric Matrix
        Output:
            E (float): Energy of the System
    """
    
    K_E = (np.linalg.multi_dot((vel.T, M, vel))) / 2
    
    U_E = 0
    for i in range(M.shape[0]):
        for j in range(i, M.shape[0]):
            if i == j:
                U_E += K[i][i] * pos[i][0]**2
            if i != j:
                U_E += K[i][j] * (pos[i][0] - pos[j][0])**2
    U_E = U_E / 2
    
    E = U_E + K_E

    return E

def post_process_final_state(final_state, B, N, y_0):
    """
        Definition:
            Find the final position and velocity vectors
        Args:
            final_state (nd_array 1x2*N^2): Normalized final state taking phases into account
            B (nd_array N^2xN^2): Padded B Matrix 
            N (int): Number of masses - should be 2^n
            y_0 (nd_array 1xN): Initial Position of Masses in Y Coordinates
        Output:
            results (dict): Dictionary containing final position and velocity vectors
    """

    ydot_final = final_state[:N]

    last_half = final_state[N**2:]

    if np.linalg.matrix_rank(B) == B.shape[0] or np.linalg.matrix_rank(B) == B.shape[1]:
        y_final = np.linalg.solve(B, last_half)[:N]
    else:
        y_final_0_padded = np.linalg.lstsq(B.T,-1j*last_half)[0]
        
        U, S, Vt = np.linalg.svd(B.T)
        rank = np.sum(S > 1e-10)
        null_space = Vt[rank:].T
        null_space_reduced = null_space[:, -1]
        null_space_reduced = null_space_reduced[..., None]

        y_0sum = np.sum(y_0)
        def pos_sum(c):
            
            y_guess = (c * null_space_reduced[:N] + y_final_0_padded[:N]).real

            return abs(np.sum(y_guess) - y_0sum)
        
        c_optimized = scipy.optimize.minimize(pos_sum, x0=0)

        y_final = c_optimized.x * null_space_reduced[:N] + y_final_0_padded[:N]
         
    return {"y_final": y_final, "ydot_final": ydot_final}
        
def calculate_error(x_quantum, xdot_quantum, x_classic, xdot_classic, N):
    """
        Definition:
            Compare the signs and amplitudes of the final solutions
        Args:
            x_quantum (array Nx1): Final positions taken from quantum solution
            xdot_quantum (array Nx1): Final velocities taken from quantum solution
            x_classical (1D list N): Final positions taken from classical solution
            xdot_classical (1D list N): Final velocities taken from classical solution
            N (int): Number of masses - should be 2^n
        Output:
            (dict): Dictionary containing sign errors and amplitude errors
                errors (dict): Error percent for each value
                sign_check (dict): Sign check for each value
    """

    errors = {}
    sign_check = {}

    for i in range(1, N+1):
        errors.update({f"Position of Mass {i}": 0})
        errors.update({f"Velocity of Mass {i}": 0})
        sign_check.update({f"Position of Mass {i}": "-"})
        sign_check.update({f"Velocity of Mass {i}": "-"})

    for i in range(N):
        errors[f"Position of Mass {i + 1}"] = (abs(abs(x_quantum[i]) - abs(x_classic[i])) / abs(x_classic[i]) * 100)[0]
        errors[f"Velocity of Mass {i + 1}"] = (abs(abs(xdot_quantum[i]) - abs(xdot_classic[i])) / abs(xdot_classic[i]) * 100)[0]

        if np.sign(x_quantum[i]) == np.sign(x_classic[i]):
            sign_check[f"Position of Mass {i+1}"] = "+"
        if np.sign(xdot_quantum[i]) == np.sign(xdot_classic[i]):
            sign_check[f"Velocity of Mass {i+1}"] = "+"

    return {"amplitude_error": errors, "sign_error": sign_check}

def back_coordinate_transformation(M, y_final, ydot_final):
    """
        Definition:
            Transform y coordinates to x coordinates
        Args:
            M (nd_array NxN): Mass Matrix - Diagonal Matrix
            y_final (nd_array 1xN): Final Positions in y coordinates
            ydot_final (nd_array 1xN): Final Velocities in y coordinates
        Outputs:
            X (dict): Dictionary containing transformed coordinates    
                x_final (nd_array 1xN): Transformed Positions
                xdot_final (nd_array 1xN): Transformed Velocities        
    """

    x_final = np.matmul(np.linalg.inv(np.sqrt(M)), y_final)
    xdot_final = np.matmul(np.linalg.inv(np.sqrt(M)), ydot_final)

    X = {"x_final": x_final, "xdot_final": xdot_final}

    return X

### Post-Processing

- We obtain the final velocity and position vectors
- We make a comparison between classical and quantum results

In [27]:
def sum_amplitudes_by_state(simulated_states):
    """
        Definition:
            Sum all the states that corresponds to same output qnum
        Args:
            simulated_states (list): Contains the state vector simulation results
        Output:
            state_amplitude_sums (dict): Amplitude and phase of each output qnum
    """
    state_amplitude_sums = {state: 0 for state in range(len(simulated_states))}
    
    for sim_state in simulated_states:
        state_value = sim_state.state['state']
        if state_value in state_amplitude_sums:
            state_amplitude_sums[state_value] += sim_state.amplitude
    
    return state_amplitude_sums

def normalize_final_state(state_amplitudes, parsed_state_vector):
    """
        Definition:
            Normalize the final state
        Args:
            state_amplitude_sums (dict): Amplitude and phase of each output qnum
            parsed_state_vector (dict): Results of the state vector simulation
        Output:
            final_state (nd_array 2*N^2x1): Normalized final state taking phases into account
    """

    values = np.array([state_amplitudes[i] for i in range(len(state_amplitudes))])
    global_phase = np.angle(values)[0]
    values = values / np.exp(1j * global_phase)

    normalization = np.linalg.norm(
            [
                parsed_state.amplitude
                for parsed_state in parsed_state_vector
            ]
        )

    final_state = values / normalization

    return final_state


def simplify_final_state(final_state):
    """
        Definition:
            Normalize the final state
        Args:
            final_state (nd_array 2*N^2x1): Normalized final state taking phases into account
        Output:
            simplified_final_state (nd_array 2*N^2x1): Final state with rounding
    """
    
    epsilon = 10**(-4)

    simplified_final_state = final_state
    for i, value in enumerate(final_state):
        imag = np.imag(value)
        real = np.real(value)
        
        if abs(np.real(value)) < epsilon:
            real = 0
        if abs(np.imag(value)) < epsilon:
            imag = 0
        simplified_final_state[i] = real + 1j*imag

    return simplified_final_state


def post_process_final_state(final_state, B, N, y_0):
    """
        Definition:
            Find the final position and velocity vectors
        Args:
            final_state (nd_array 1x2N^2): Normalized final state taking phases into account
            B (nd_array N^2xN^2): Padded B Matrix 
            N (int): Number of masses - should be 2^n
            y_0 (nd_array 1xN): Initial Position of Masses in Y Coordinates
        Output:
            results (dict): Dictionary containing final position and velocity vectors
    """

    ydot_final = final_state[:N]

    last_half = final_state[N**2:]

    if np.linalg.matrix_rank(B) == B.shape[0] or np.linalg.matrix_rank(B) == B.shape[1]:
        y_final = np.linalg.solve(B, last_half)[:N]
    else:
        y_final_0_padded = np.linalg.lstsq(B.T,-1j*last_half)[0]

        U, S, Vt = np.linalg.svd(B.T)
        rank = np.sum(S > 1e-10)
        null_space = Vt[rank:].T
        null_space_reduced = null_space[:, -1]
        null_space_reduced = null_space_reduced[..., None]

        y_0sum = np.sum(y_0)
        def pos_sum(c):

            y_guess = (c * null_space_reduced[:N] + y_final_0_padded[:N]).real

            return abs(np.sum(y_guess) - y_0sum)

        c_optimized = scipy.optimize.minimize(pos_sum, x0=0)

        y_final = c_optimized.x * null_space_reduced[:N] + y_final_0_padded[:N]

    return {"y_final": y_final, "ydot_final": ydot_final}

def calculate_error(y_quantum, ydot_quantum, y_classic, ydot_classic, N):
    """
        Definition:
            Compare the signs and amplitudes of the final solutions
        Args:
            y_quantum (array Nx1): Final positions taken from quantum solution
            ydot_quantum (array Nx1): Final velocities taken from quantum solution
            y_classical (1D list N): Final positions taken from classical solution
            ydot_classical (1D list N): Final velocities taken from classical solution
            N (int): Number of masses - should be 2^n
        Output:
            (dict): Dictionary containing sign errors and amplitude errors
                errors (dict): Error percent for each value
                sign_check (dict): Sign check for each value
    """

    errors = {}
    sign_check = {}

    for i in range(1, N+1):
        errors.update({f"Position of Mass {i}": 0})
        errors.update({f"Velocity of Mass {i}": 0})
        sign_check.update({f"Position of Mass {i}": "-"})
        sign_check.update({f"Velocity of Mass {i}": "-"})

    for i in range(N):
        errors[f"Position of Mass {i + 1}"] = (abs(abs(y_quantum[i]) - abs(y_classic[i])) / abs(y_classic[i]) * 100)[0]
        errors[f"Velocity of Mass {i + 1}"] = (abs(abs(ydot_quantum[i]) - abs(ydot_classic[i])) / abs(ydot_classic[i]) * 100)[0]

        if np.sign(y_quantum[i]) == np.sign(y_classic[i]):
            sign_check[f"Position of Mass {i+1}"] = "+"
        if np.sign(ydot_quantum[i]) == np.sign(ydot_classic[i]):
            sign_check[f"Velocity of Mass {i+1}"] = "+"

    return {"amplitude_error": errors, "sign_error": sign_check}

#### Pre-Processing of simple case $1$ for quantum evolution:

- Create B Matrix, Hamiltonian and initial state

In [5]:
N, M, K, x_0, xdot_0 = initialize_system() # plug N=2 for 2 case system 

# F Matrix
F= create_matrix_F(K, N)

# A Matrix
A = create_matrix_A(M, F)

# Transform coordinates
y = coordinate_transformation(M, x_0, xdot_0)
y_0 = y["y_0"]
ydot_0 = y["ydot_0"]

# B Matrix
B = create_matrix_B(M, K, A, N)

# Stacking B with zeros to get square matrix N^2 x N^2
B_padded = padding_B(B, N)

# Hamiltonian
Ham = create_Hamiltonian(B_padded)

# Initial State
init_state = create_init_state(B_padded, y_0, ydot_0, N)
E0_y = calculate_energy(y_0, ydot_0, M, K)

# Normalize the Initial State
normalization = normalize_init_state(init_state)
normalized_init_state = normalization["normalized_init_state"]
norm = normalization["norm"]

# Print the results

print("Number of masses", N)
print("Mass matrix", M)
print("Spring Constant Matrix", K)
print("Initial Energy: ", E0_y)
print("B matrix: ", B_padded)
print("Hamiltonian formed:", Ham)
print("Initial State:", init_state)

assert np.matmul(B, B.conj().T).all()==A.all()

Number of masses 2
Mass matrix [[2. 0.]
 [0. 1.]]
Spring Constant Matrix [[1. 2.]
 [2. 3.]]
Initial Energy:  [[9.84314575]]
B matrix:  [[ 0.70710678+0.j  1.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j -1.41421356+0.j  1.73205081+0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]
 [ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j]]
Hamiltonian formed: [[-0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j
  -0.70710678-0.j -1.        -0.j -0.        -0.j -0.        -0.j]
 [-0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j
  -0.        -0.j  1.41421356-0.j -1.73205081-0.j -0.        -0.j]
 [-0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j
  -0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j]
 [-0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j
  -0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j]
 [-0.70710678+0.j -0.        +0.j -0.       

In [6]:
np.transpose(Ham)==Ham    # H is hermitian (and all entries are real), this ensures we can use qubitization and product formula based simulations

array([[ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True]])

- Finding the Pauli decomposition of the Hamiltonian formed above:

(Below functions for Pauli Decomposition is taken from [Classiq Library](https://github.com/Classiq/classiq-library/blob/main/algorithms/hhl/hhl/hhl.ipynb), thanks a lot to the authors!). This method as we saw, works fairly well for smaller Hamiltonians. 

In [7]:
def generate_all_pauli_strings(paulis, n):
    """
        Definition:
            Create all combination of Pauli Matrices for n qubits
        Args:
            paulis (string): Name of the Pauli Matrices
            n (int): log2(2*N^2) where N is the number of masses
        Outputs:
            pauli_comb (list of strings): All combination of Pauli Matrices
    """    
    for s in itertools.product(paulis, repeat=n):
        yield "".join(s)

def pauli_string_2mat(seq):
    """
        Definition:
            Transform a Pauli Matrix Sequence to a single Matrix
        Args:
            seq (string): Name of the Pauli Matrices as a sequence
        Outputs:
            pauli_mat (nd_array): Multipication of Pauli Matrices
    """ 
    pauli_mat = PAULI_DICT[seq[0]]
    for p in seq[1:]:
        pauli_mat = np.kron(pauli_mat, PAULI_DICT[p])
    return pauli_mat

def hilbert_schmidt(M1, M2):
    """
        Definition:
            Hilbert-Schmidt Product of two matrices
        Args:
            M1 (nd_array): Matrix 1
            M2 (nd_array): Matrix 2
        Outputs:
            (nd_array): Product result
    """ 
    return (np.dot(M1.conjugate().transpose(), M2)).trace()

def lcu_naive(Ham):
    """
        Definition:
            Create the Pauli-Matrix-Decomposition of Hamiltonian
        Args:
            H (nd_array 2*N^2x2*N^2): Padded Hamiltonian - Hermitian Matrix
        Outputs:
            pauli_list (list): Each Pauli Matrix sequence with non-zero coefficient
    """

    # assert Ham.shape[0] == Ham.shape[1], "Matrix is not square"
    # assert Ham.shape[0] != 0, "Matrix is of size 0"
    # assert Ham.shape[0] & (Ham.shape[0] - 1) == 0, "Matrix size is not 2**n"

    n = int(np.log2(Ham.shape[0]))
    possible_pauli_list = list(generate_all_pauli_strings("IZXY", n))

    pauli_list = []

    for pstr in possible_pauli_list:
        co = (1 / 2**n) * hilbert_schmidt(pauli_string_2mat(pstr), Ham)
        if co != 0:
            pauli_list = pauli_list + [(pstr, co)]

    return pauli_list

def pauli_str_to_enums(pauli):
    """
        Definition:
            Create Classiq Pauli matrices for Pauli Matrix Sequence
        Args:
            pauli (string): Pauli Matrix Sequence String
        Outputs:
            (list): Classiq Pauli Matrices
    """
    return [CHAR_TO_STUCT_DICT[s] for s in pauli]

def pauli_list_to_hamiltonian(pauli_list):
    """
        Definition:
            Create complete Pauli list for a given Hamiltonian
        Args:
            pauli_list (list): Each Pauli Matrix sequence with non-zero coefficient
        Outputs:
            (list): Classiq PauliTerm function with appropriate Pauli matrices
    """
    return [
        PauliTerm(
            pauli=pauli_str_to_enums(pauli), coefficient=typing.cast(complex, coeff).real
        )
        for pauli, coeff in pauli_list
    ]

In [8]:
# Create the Pauli Matrix Decomposition of the Hamiltonian
pauli_list = lcu_naive(Ham)

# Transform Pauli Matrix Decomposition to Classiq compatible
classiq_pauli_list = pauli_list_to_hamiltonian(pauli_list)

print(pauli_list)

[('XII', (0.17677669529663692+0j)), ('XIZ', (-0.5303300858899107+0j)), ('XIX', (-0.25+0j)), ('XZI', (0.17677669529663692+0j)), ('XZZ', (-0.5303300858899107+0j)), ('XZX', (-0.25+0j)), ('XXX', (-0.4330127018922193+0j)), ('XYY', (-0.4330127018922193+0j)), ('YIY', (0.25+0j)), ('YZY', (0.25+0j)), ('YXY', (-0.4330127018922193+0j)), ('YYX', (0.4330127018922193+0j))]


In [9]:
# check if the decomposition is correct, we use qutip for this

# Define the Pauli matrices and the identity matrix
pauli_operators = {
    'I': qt.qeye(2),
    'X': qt.sigmax(),
    'Y': qt.sigmay(),
    'Z': qt.sigmaz()
}

def create_hamiltonian(hamiltonian_list):
    hamiltonian = 0
    num_qubits = len(hamiltonian_list[0][0])  

    for term in hamiltonian_list:
        pauli_string, coefficient = term
        
        # calculating tensor products
        term_operator = pauli_operators[pauli_string[0]]
        
        
        for pauli in pauli_string[1:]:
            term_operator = qt.tensor(term_operator, pauli_operators[pauli])
        
        # Adding the terms of the Hamiltonian
        hamiltonian += coefficient * term_operator

    return hamiltonian

hamiltonian = create_hamiltonian(pauli_list)

print(hamiltonian)



Quantum object: dims=[[2, 2, 2], [2, 2, 2]], shape=(8, 8), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.          0.          0.          0.         -0.70710678 -1.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          1.41421356
  -1.73205081  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]
 [-0.70710678  0.          0.          0.          0.          0.
   0.          0.        ]
 [-1.          1.41421356  0.          0.          0.          0.
   0.          0.        ]
 [ 0.         -1.73205081  0.          0.          0.          0.
   0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.        ]]


Thus as we see, the decomposition is exact.

### Hamiltonian simulation

We use the `exponentiation_with_depth_constraint` function of classiq that produces a quantum gate that approximates $e^{iHt}$, for any input **Hermitian operator** $H$ . Since the Hamiltonian for this algorithm is Hermitian by definition, exponentian method can be used straightforwardly. The Classiq engine automatically generates an efficient higher-order Trotter-Suzuki quantum program that minimizes the functional error and satisfies a given local constraint on the depth. 

In [28]:
### Define function for Quantum Algorithm

@qfunc
def init_phase(state: QNum):
    """
        Definition:
            Imply -pi/2 phase to the last half (msb qubit) of the QNum state in order to add -i factor
        Args:
            state (QNum): Initial state with bare amplitudes
        Outputs:
            state (QNum): Initial state with phase
    """

    state_in_qubit = QArray("state_in_qubit")
    msb = QArray("msb", QBit)
    
    size = np.log2(init_state.size)
    allocate(size, msb)

    bind(state, state_in_qubit)
    repeat(state_in_qubit.len, lambda i: CX(state_in_qubit[i], msb[i]))
    control(msb[size-1], lambda: PHASE(np.pi/2, state_in_qubit[size-1]))
    bind(state_in_qubit, state)




@qfunc
def main(time: CReal, state: Output[QNum]):
    """
        Definition:
            Main function to make evolution

        Outputs:
            state (Output[QNum]): Resulting state at the t = evolution_coefficient
    """
        
    prepare_amplitudes(amplitudes=list(normalized_init_state), out=state, bound=0.000000000001)
    init_phase(state)
    # exponentiation_with_depth_constraint(
    #     classiq_pauli_list,
    #     evolution_coefficient=0.5,
    #     max_depth=1000,
    #     qbv=state)
    
    suzuki_trotter(
        classiq_pauli_list,
        evolution_coefficient=time,
        order=1,
        repetitions=10,
        qbv=state,
    )

model = create_model(main)


In [29]:
Execution_Prefs = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    ),
)

In [ ]:
# def simulation():
#     qprog= create_model(main)
#     qmod= synthesize(qprog)
#     # Execute the program and get parsed results
#     job = execute(qprog)
#     parsed_state_vector= job.result()[0].value.parsed_state_vector  # to get the phases
#     # Find the meausurement results of state in a dictionary format
#     state_amplitudes = sum_amplitudes_by_state(parsed_state_vector)
#     # Normalize the final state
#     normalized_final_state = normalize_final_state(state_amplitudes, parsed_state_vector)
#     # Simplify the final state by neglecting small terms
#     simplified_final_state = simplify_final_state(normalized_final_state)
#     final_state = norm * simplified_final_state[..., None]
#     # print("The final quantum state from quantum evolution", final_state)
#     final_results = post_process_final_state(final_state, B_padded, N, 0.5/0.707)
#     y_final = final_results["y_final"]
#     ydot_final = final_results["ydot_final"]
#     return y_final, ydot_final 


# total_time = 1
# steps= 10

# # Initialize lists to store the data points
# x_vals = list(range(steps))
# y1_vals, y2_vals, y3_vals, y4_vals = [], [], [], []

# # Create subplots
# fig, axs = plt.subplots(2, 2, figsize=(10, 8))

# # Set titles for the subplots
# axs[0, 0].set_title("Plot 1")
# axs[0, 1].set_title("Plot 2")
# axs[1, 0].set_title("Plot 3")
# axs[1, 1].set_title("Plot 4")


# for time in np.linspace(0,total_time,steps):
#     final_pos, final_vel= simulation()

#     # Append the values to the respective lists
#     y1_vals.append(final_pos[0, 0])
#     y2_vals.append(final_pos[1, 0])
#     y3_vals.append(final_vel[0, 0])
#     y4_vals.append(final_vel[1, 0])
    
#     # Clear previous plots
#     axs[0, 0].cla()
#     axs[0, 1].cla()
#     axs[1, 0].cla()
#     axs[1, 1].cla()
    
#     # Plot the new data points
#     axs[0, 0].plot(x_vals[:time+1], y1_vals, 'b-')
#     axs[0, 1].plot(x_vals[:time+1], y2_vals, 'r-')
#     axs[1, 0].plot(x_vals[:time+1], y3_vals, 'g-')
#     axs[1, 1].plot(x_vals[:time+1], y4_vals, 'k-')
    
#     # Redraw the plots
#     plt.pause(0.01)  # Pause to update the plots

# plt.ioff()  # Turn off interactive mode
# plt.show()  # Show the final plot

In [30]:
# Execute the Quantum Algorithm with state vector simulation since phase is important
# backend_preferences = ClassiqBackendPreferences(backend_name="simulator_statevector")
# model_pref = set_execution_preferences(model, ExecutionPreferences(num_shots=1, backend_preferences=backend_preferences))
# qprog = synthesize(model_pref)
# #show(qprog)

total_time = 1
steps= 5

qprog = synthesize(model)
es = ExecutionSession(qprog, execution_preferences=Execution_Prefs)
for i in np.linspace(0,total_time,steps):
    result = es.sample({"time": i})
    parsed_state_vector= result.parsed_state_vector
    state_amplitudes = sum_amplitudes_by_state(parsed_state_vector)
    # Normalize the final state
    normalized_final_state = normalize_final_state(state_amplitudes, parsed_state_vector)

    # Simplify the final state by neglecting small terms
    simplified_final_state = simplify_final_state(normalized_final_state)

    # Correct the normalization factor that comes from the normalization of initial state and transform the row vector to column vector
    final_state = norm * simplified_final_state[..., None]
    print("The final quantum state from quantum evolution", final_state)

    # Get the final position and velocity vectors
    final_results = post_process_final_state(final_state, B_padded, N, y_0)
    y_final = final_results["y_final"]
    ydot_final = final_results["ydot_final"]

    # transform to original coordinates

    x_final_results = back_coordinate_transformation(M, y_final, ydot_final)
    x_final = x_final_results["x_final"]
    xdot_final = x_final_results["xdot_final"]

    print("Final Positions", x_final)
    print("Final Velocities", xdot_final)

    


The final quantum state from quantum evolution [[1.41421356+0.j        ]
 [1.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +1.j        ]
 [0.        -1.41421356j]
 [0.        +3.46410162j]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j 

LinAlgError: Incompatible dimensions

In [18]:
# # Execute the program and get parsed results
# job = execute(qprog)
# parsed_state_vector= job.result()[0].value.parsed_state_vector  # to get the phases
# parsed_state_vector


[SimulatedState(state={'state': 0.0}, bitstring='000000', amplitude=(-0.46971611518808354+0.46971611518810846j)),
 SimulatedState(state={'state': 1.0}, bitstring='001001', amplitude=(-0.3107230047287697+0.3107230047287946j)),
 SimulatedState(state={'state': 5.0}, bitstring='101101', amplitude=(-0.289309064494346-0.2893090644943128j)),
 SimulatedState(state={'state': 5.0}, bitstring='001101', amplitude=(0.1659338632294986+0.16593386322947953j)),
 SimulatedState(state={'state': 1.0}, bitstring='101001', amplitude=(-0.1659338632294807+0.16593386322949694j)),
 SimulatedState(state={'state': 5.0}, bitstring='000101', amplitude=(-0.16592901030462356-0.16592901030461107j)),
 SimulatedState(state={'state': 0.0}, bitstring='101000', amplitude=(0.11732952838196081-0.11732952838197408j)),
 SimulatedState(state={'state': 1.0}, bitstring='000001', amplitude=(-0.04282913322252821+0.04282913322252233j)),
 SimulatedState(state={'state': 0.0}, bitstring='001000', amplitude=(-0.030284770533991454+0.0302

In [19]:
# Find the meausurement results of state in a dictionary format
state_amplitudes = sum_amplitudes_by_state(parsed_state_vector)
state_amplitudes

{0: (-0.3826713573401142+0.38267135734012375j),
 1: (-0.5194860011807786+0.5194860011808139j),
 2: (3.6190685009668237e-16-4.4276524994367796e-16j),
 3: (3.7087336276834813e-16-4.337973608189734e-16j),
 4: (-4.1026830174013795e-06-4.102683022455948e-06j),
 5: (-0.28930421156947095-0.28930421156944436j),
 6: (1.6715437659022589e-16+1.9366874205961702e-16j),
 7: (1.888049741258425e-16+1.7202121416018337e-16j)}

In [20]:
# Normalize the final state
normalized_final_state = normalize_final_state(state_amplitudes, parsed_state_vector)
normalized_final_state

# Simplify the final state by neglecting small terms
simplified_final_state = simplify_final_state(normalized_final_state)
simplified_final_state

# Correct the normalization factor that comes from the normalization of initial state and transform the row vector to column vector
final_state = norm * simplified_final_state[..., None]
print("The final quantum state from quantum evolution", final_state)



The final quantum state from quantum evolution [[1.08235805+0.j        ]
 [1.4693283 +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.81827588j]
 [0.        +0.j        ]
 [0.        +0.j        ]]


In [21]:
H_qutip = qt.Qobj(Ham)
rho0 = qt.Qobj(init_state)
tlist = np.linspace(0,0.5,100)
results1 = qt.mesolve(H_qutip,rho0, tlist,[],[])
results1.states[-1]

Quantum object: dims=[[8], [1]], shape=(8, 1), type='ket', dtype=Dense
Qobj data =
[[1.08234647+0.j        ]
 [1.46933095+0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.81828735j]
 [0.        +0.j        ]
 [0.        +0.j        ]]

In [22]:

# Get the final position and velocity vectors
final_results = post_process_final_state(final_state, B_padded, N, y_0)
y_final = final_results["y_final"]
ydot_final = final_results["ydot_final"]

# transform to original coordinates

x_final_results = back_coordinate_transformation(M, y_final, ydot_final)
x_final = x_final_results["x_final"]
xdot_final = x_final_results["xdot_final"]

print("Final Positions", x_final)
print("Final Velocities", xdot_final)


Final Positions [[ 1.08925521+0.j]
 [-0.54551725+0.j]]
Final Velocities [[0.76534271+0.j]
 [1.4693283 +0.j]]


### Classical Numerical Solution:

In [24]:
def equations_of_motion(t, init, N, M, K):
    """
    Definition:
        EOM for classical coupled oscillator system with N masses
    Args:
        t (array): Time array
        init (list): Initial Conditions
        N (int): Number of masses
        M (array): Mass matrix (diagonal)
        K (array): Spring constant matrix
    Output:
        (list): List of final equations
    """
    x = init[:N]
    v = init[N:]
    dxdt = v
    dvdt = np.zeros(N)
    
    for i in range(N):
        if i == 0:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i+1] * (x[i+1] - x[i])) / M[i, i]
        elif i == N-1:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i-1] * (x[i-1] - x[i])) / M[i, i]
        else:
            dvdt[i] = (-K[i, i] * x[i] + K[i, i-1] * (x[i-1] - x[i]) + K[i, i+1] * (x[i+1] - x[i])) / M[i, i]

    return np.concatenate((dxdt, dvdt))


# Initial conditions: [x1(0), x2(0), ..., xN(0), v1(0), v2(0), ..., vN(0)]
initial_conditions = np.concatenate((x_0, xdot_0))


# Time span for the simulation
t_span = (0, 0.5)
t_eval = np.linspace(t_span[0], t_span[1], 100)

# Solve the system of differential equations
solution = scipy.integrate.solve_ivp(equations_of_motion, t_span, initial_conditions.flatten(), t_eval=t_eval, args=(N, M, K))

# Extract the results
t = solution.t
x = solution.y[:N, :]
v = solution.y[N:, :]


# # Plot position results
# plt.figure(figsize=(10, 6))
# for i in range(N):
#     plt.plot(t, x[i], label=f'x{i+1} (Position of Mass {i+1})')
# plt.xlabel('Time')
# plt.ylabel('Position')
# plt.legend()
# plt.title('Coupled Harmonic Oscillators (Numerical Solution)')
# plt.grid()
# plt.show()

# # Plot velocity results
# plt.figure(figsize=(10, 6))
# for i in range(N):
#     plt.plot(t, v[i], label=f'v{i+1} (Velocity of Mass {i+1})')
# plt.xlabel('Time')
# plt.ylabel('Velocity')
# plt.legend()
# plt.title('Velocities of Coupled Harmonic Oscillators (Numerical Solution)')
# plt.grid()
# plt.show()


# Get the results numerically
x_classic =np.array([x[0][-1], x[1][-1]])
xdot_classic =np.array([v[0][-1], v[1][-1]])

print("Position Vectors at t=0.5 classically")
print(x_classic)
print("Velocity Vectors at t=0.5 classically")
print(xdot_classic)


Position Vectors at t=0.5 classically
[1.43942859 0.62114283]
Velocity Vectors at t=0.5 classically
[0.76533463 1.46933074]


### Comparision between Classical and Quantum Solutions

In [79]:

# Compare two solutions
comparision = calculate_error(y_final, ydot_final, x_classic, xdot_classic, N)
amplitude_error = comparision["amplitude_error"]
sign_error = comparision["sign_error"]

print("Percentatage error in positions and velocities", amplitude_error)
print(sign_error)

Percentatage error in positions and velocities {'Position of Mass 1': 64.75489192488729, 'Velocity of Mass 1': 98.07530661416715, 'Position of Mass 2': 93.42917068061023, 'Velocity of Mass 2': 304.06846556503024}
{'Position of Mass 1': '-', 'Velocity of Mass 1': '+', 'Position of Mass 2': '-', 'Velocity of Mass 2': '+'}


### Hamiltonian Simulation using block-encoding:
___

#### Block encoding H

We have our hamiltonian of the form 
$$
H = \sum^{L-1}_{i=0} \alpha_i U_i,
$$
where $U_i$ are Pauli strings. We use Linear Combination of Unitaries (LCU) to form the block-encoding of $H$

LCU refers to $(\bar{\alpha}, m, 0)$-block encoding of a matrix given as a sum of unitaries:
$$
U_{(\bar{\alpha},m,0)-H} =\begin{pmatrix}
H/\bar{\alpha} & * \\
* & *
\end{pmatrix}, \text{   for     } H = \sum^{L-1}_{i=0} \alpha_i U_i, \,\,\,\,\, \alpha_i\geq 0
$$
with $\bar{\alpha}\equiv\sum^{L-1}_{i=0}\alpha_i$ and $m= \lceil\log_2(L) \rceil$ represents the numner of block qubits. 


More details on the LCU methodology can be found in classiq's [LCU tutorial](https://github.com/Classiq/classiq-library/blob/main/tutorials/linear_combination_of_unitaries/linear_combination_of_unitaries.ipynb)
___

#### Block encoding $e^{iHt}$

Given block-encoding of the Hamiltonian $H$, we can define the Szegedy quanutm walk-operator as: 
$$
W\equiv -\Pi_{|0\rangle_m} U_{(\bar{\alpha},m,0)-H},
$$  
where $\Pi_{|0\rangle_m}$ is a reflection operator about the block state. The powers of $W$ correspond to a $(1,m,0)$ block encoding of the Chebyshev polynomials of the matrix $H$:
$$
W^k = \begin{pmatrix}
T_k(H) & * \\
* & *
\end{pmatrix}=U_{(1,m,0)-T_k(H)},
$$
with $T_k$ being the k-th Chebyshev polynomial. Its useful to note that this property holds only for an **Hermitian block-encoding**. From the Jacobi -Anger expansion (given below) we can have an $\epsilon$-approximation of $\exp(iHt)\approx \sum^d_{i=0} \beta_{i} T_{i}(H)$, for which we can perform the following encoding
$$
U_{(\bar{\beta},\tilde{m},\epsilon)-\exp{(iHt)}} =
\begin{pmatrix}
\exp{(iHt)}/\bar{\beta} & * \\
* & *
\end{pmatrix}=
% \begin{pmatrix}
% \frac{1}{\bar{\beta}}\sum^d_{k=0} \beta_{k} T_{k}(Ht) & * \\
% * & *
% \end{pmatrix}=
\begin{pmatrix}
\frac{1}{\bar{\beta}}\sum^d_{k=0} \beta_{k} W^k & * \\
* & *
\end{pmatrix}
% \begin{pmatrix}
% \frac{1}{\bar{\beta}}\sum^d_{k=0} W^k & * \\
% * & *
% \end{pmatrix},
$$
where $\tilde{m}=m+\lceil \log_2(d+1) \rceil$ 

We evaluate the coefficients $\beta_{i}$ as the bessel coefficients ($J(t)$) of the Jacobi-anger expressions for sine and cosine as follows
$$
\cos(Ht)= J_0(t) + 2\sum^{d}_{k=1} (-1)^k J_{2k}(t) T_{2k}(H)\\
\sin(Ht)= 2\sum^{d}_{k=0} (-1)^k J_{2k+1}(t) T_{2k+1}(H),
$$


___

#### **Block Encoding the Hamiltonian (LCU):**

##### Preparing the normalized coefficients for the PREPARE operation

The PREPARE operation prepares a quantum state that corresponds to the probabilities $|\alpha_i| /\sum|\alpha_i|$. We define a function that gets the list of  coefficients of the Hamiltonian and returns the probabilities to be loaded as part of the PREPARE operation. The block size is also determined based on the number of terms in the Hamiltonian.

In [25]:
# getting the normalized coefficents representing probabilities for the prepare block of LCU

def get_normalized_lcu_coef(lcu_coef):
    # Calculate the normalization factor as the sum of moduli of the complex numbers
    normalization_factor = sum(np.abs(c) for c in lcu_coef)
 
    # Calculate the prepare_prob list as specified
    prepare_prob = [(np.abs(c) / normalization_factor) for c in lcu_coef]
    
    # Print the prepare_prob list
    print("The prepared probabilities:", prepare_prob)
    
    # Calculate the size of the block encoding
    coef_size = int(np.ceil(np.log2(len(prepare_prob))))
    
    # Pad prepare_prob with zeros to make its length a power of 2
    prepare_prob += [0] * (2**coef_size - len(prepare_prob))

    if sum(prepare_prob)>1:
        k= sum(prepare_prob)-1 
        prepare_prob[0]-=k
    
    if sum(prepare_prob)<1:
        k= 1- sum(prepare_prob) 
        prepare_prob[0]+=k
    
    # Print the details
    print("The size of the block encoding:", coef_size)
    print("The normalized coefficients:", prepare_prob)
    print("The normalization factor:", normalization_factor)
    
    return normalization_factor, coef_size, prepare_prob

lcu_pauli_coef = [p.coefficient for p in classiq_pauli_list]

normalization_ham, lcu_size_ham, prepare_probs_ham = get_normalized_lcu_coef(
    lcu_pauli_coef
)
normalization_ham

The prepared probabilities: [0.1464466094067262, 0.1464466094067262, 0.10355339059327373, 0.1464466094067262, 0.1464466094067262, 0.10355339059327373, 0.10355339059327373, 0.10355339059327373]
The size of the block encoding: 3
The normalized coefficients: [0.14644660940672644, 0.1464466094067262, 0.10355339059327373, 0.1464466094067262, 0.1464466094067262, 0.10355339059327373, 0.10355339059327373, 0.10355339059327373]
The normalization factor: 1.707106781186548


1.707106781186548

#### Defining a function that applies the $(PREPARE) SELECT (PREPARE^{\dagger})$ operation

The SELECT operation acts on the desired target state controlled by the quantum state of the block qubits. This constitution ends our construction of the unitary block Hamiltonian. 

In [26]:
@qfunc
def apply_pauli_term(pauli_string: PauliTerm, x: QArray[QBit]):
    repeat(
        count=x.len,
        iteration=lambda index: switch(
            pauli_string.pauli[index],
            [
                lambda: IDENTITY(x[pauli_string.pauli.len - index - 1]),
                lambda: X(x[pauli_string.pauli.len - index - 1]),
                lambda: Y(x[pauli_string.pauli.len - index - 1]),
                lambda: Z(x[pauli_string.pauli.len - index - 1]),
            ],
        ),
    )
    
@qfunc
def lcu_paulis(
    pauli_terms_list: CArray[PauliTerm],
    probs: CArray[CReal],
    block: QNum,
    data: QArray[QBit],
):
    within_apply(
        lambda: inplace_prepare_state(probs, 0.0, block),
        lambda: repeat(
            count=pauli_terms_list.len,
            iteration=lambda i: control(
                block == i, lambda: apply_pauli_term(pauli_terms_list[i], data)
            ),       
            ),
    )
      

#### **Block Encoding the Hamiltonian evolution ($e^{iHt}$): (Qubitization method)**
##### Defining the walk-operator $W$



In [27]:
@qfunc
def my_walk_operator(block: QArray[QBit], data: QArray[QBit]) -> None:
    lcu_paulis(classiq_pauli_list, prepare_probs_ham, block, data)
    reflect_about_zero(block)
    RY(2 * np.pi, block[0])  # for the minus sign

##### Evaluating the Chebyshev co-efficients

We calculate the coefficients $\beta_i$ for approximating the sine and cosine functions. We want to approximate $e^{iHt}$, but our Hamiltonian is encoded with a normalization factor $\bar{\alpha}$ that comes from the LCU block encoding. Therefore, we rescale the time by this factor. 

Given the coefficients, we calculate the probabilities for the LCU prepare variable. These must be positive numbers. Moreover, for the odd terms, which corresponds to the sine function, we should add a factor of $i$. Those factors should be added to the unitary operations $W^k$. To take this into account, for each term we define a "generalized sign" $\sigma_k$ such that $\beta_k = e^{\frac{\pi}{2}\sigma_k i} |\beta_k|$ with $\sigma_k \in \{0,1,2,3\}$ that corresponds to the factors $1,i,-1,-i$ respectively. 

In [28]:
def get_cheb_coef(epsilon, t):
    poly_degree = int(
        np.ceil(
            t
            + np.log(epsilon ** (-1)) / np.log(np.exp(1) + np.log(epsilon ** (-1)) / t)
        )
    )
    cos_coef = [jv(0, t)] + [
        2 * jv(2 * k, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    sin_coef = [
        -2 * jv(2 * k - 1, t) * (-1) ** k for k in range(1, poly_degree // 2 + 1)
    ]
    return cos_coef, sin_coef


EVOLUTION_TIME = 0.5
EPS = 0.1

# since our hamiltonian from 
normalized_time = normalization_ham * EVOLUTION_TIME

cos_coef, sin_coef = get_cheb_coef(EPS, normalized_time)
combined_sin_cos_coef = []

for k in range(len(cos_coef) - 1):
    combined_sin_cos_coef.append(cos_coef[k])
    combined_sin_cos_coef.append(sin_coef[k])
combined_sin_cos_coef.append(cos_coef[-1])
if len(sin_coef) == len(cos_coef):
    combined_sin_cos_coef.append(sin_coef[-1])


signs_cheb_coef = np.sign(combined_sin_cos_coef).tolist()
generalized_signs = [
    (1 - signs_cheb_coef[s]) + (s) % 2 for s in range(len(signs_cheb_coef))
]
positive_cheb_lcu_coef = np.abs(combined_sin_cos_coef)

normalization_exp, lcu_size_exp, prepare_probs_exp = get_normalized_lcu_coef(
    positive_cheb_lcu_coef
)

The prepared probabilities: [0.4652246222700498, 0.43827721900774796, 0.09649815872220227]
The size of the block encoding: 2
The normalized coefficients: [0.4652246222700498, 0.43827721900774796, 0.09649815872220227, 0]
The normalization factor: 1.775463424856277


In [29]:
poly_degree = int(
        np.ceil(
            EVOLUTION_TIME
            + np.log(EPS ** (-1)) / np.log(np.exp(1) + np.log(EPS ** (-1)) / EVOLUTION_TIME)
        )
    )

poly_degree

2

#### Block encoding using LCU

In [30]:
@qfunc
def lcu_cheb(
    coef: CArray[CReal],
    generalized_signs: CArray[CInt],
    walk_operator: QCallable[QNum, QArray],
    walk_block: QNum,
    walk_data: QArray,
    cheb_block: QNum,
):

    within_apply(
        lambda: inplace_prepare_state(coef, 0.0, cheb_block),
        lambda: repeat(
            generalized_signs.len,
            lambda k: control(
                cheb_block == k,
                lambda: (
                    U(0, 0, 0, np.pi / 2 * generalized_signs[k], walk_data[0]),
                    power(k, lambda: walk_operator(walk_block, walk_data)),
                ),
            ),
        ),
    )

In [31]:
@qfunc
def init_state_phase(state: QNum):
    """
        Definition:
            Imply -pi/2 phase to the last half (msb qubit) of the QNum state in order to add -i factor
        Args:
            state (QNum): Initial state with bare amplitudes
        Outputs:
            state (QNum): Initial state with phase
    """

    state_in_qubit = QArray("state_in_qubit")
    msb = QArray("msb", QBit)
    
    size = np.log2(init_state.size)
    allocate(size, msb)

    bind(state, state_in_qubit)
    repeat(state_in_qubit.len, lambda i: CX(state_in_qubit[i], msb[i]))
    control(msb[size-1], lambda: PHASE(-np.pi/2, state_in_qubit[size-1]))
    bind(state_in_qubit, state)

@qfunc
def main(ham_block: Output[QNum], state: Output[QNum], exp_block: Output[QNum]):
    prepare_amplitudes(amplitudes=list(normalized_init_state), out=state, bound=0.01)
    init_state_phase(state)
    allocate(lcu_size_exp, exp_block)
    allocate(lcu_size_ham, ham_block)
    lcu_cheb(
        prepare_probs_exp,
        generalized_signs,
        lambda x, y: my_walk_operator(x, y),
        ham_block,
        state,
        exp_block,
    )
    

preferences = Preferences(
   timeout_seconds= 600
)
Execution_Prefs = ExecutionPreferences(
    num_shots=1,
    backend_preferences=ClassiqBackendPreferences(
        backend_name=ClassiqSimulatorBackendNames.SIMULATOR_STATEVECTOR
    ),
)
# constraints= Constraints(
#         optimization_parameter=OptimizationParameter.DEPTH,
#         max_width= 25
#     )

model = create_model(main, execution_preferences=Execution_Prefs)
# qmod = set_constraints(model)
qmod = set_preferences(model, preferences)


qprog = synthesize(qmod)
show(qprog)
write_qmod(qmod, "HS", decimal_precision=16)

Opening: https://platform.classiq.io/circuit/949c563f-7b3a-4b27-81aa-2cadf226c379?version=0.43.3


In [270]:
# show(qprog)

Opening: https://platform.classiq.io/circuit/e90fcc6f-0f54-4091-a91d-52d8db492c47?version=0.43.3


In [32]:
results = execute(qprog).result()


In [33]:
parsed_state_vector= results[0].value.parsed_state_vector 
parsed_state_vector

[SimulatedState(state={'ham_block': 0.0, 'state': 0.0, 'exp_block': 0.0}, bitstring='00000000000000', amplitude=(0.33528244716098676+0.10170681841795202j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 0.0, 'exp_block': 1.0}, bitstring='00000000001000', amplitude=(-0.30554533512301446-0.09268616410117422j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 5.0, 'exp_block': 0.0}, bitstring='00010100000101', amplitude=(0.08152975809334212-0.26876759331559663j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 1.0, 'exp_block': 0.0}, bitstring='00000100000001', amplitude=(0.25292404265797536+0.07672366954479215j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 1.0, 'exp_block': 1.0}, bitstring='00000100001001', amplitude=(-0.21605317842539956-0.06553901515810881j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 5.0, 'exp_block': 1.0}, bitstring='00010100001101', amplitude=(-0.06553901515810844+0.2160531784253995j)),
 SimulatedState(state={'ham_block': 0.0, 'state': 4.0, '

In [45]:
##Post Processing

def get_projected_state_vector(
    execution_result,
    measured_var: str,
    projections: dict,
) -> np.ndarray:
    """
    This function returns a reduced statevector from execution results.
    measured var: the name of the reduced variable
    projections: on which values of the other variables to project, e.g., {"ind": 1}
    """
    projected_size = len(execution_result[0].value.output_qubits_map[measured_var])
    proj_statevector = np.zeros(2**projected_size).astype(complex)
    for sample in execution_result[0].value.parsed_state_vector:
        if all(sample.state[key] == projections[key] for key in projections.keys()):
            proj_statevector[int(sample.state[measured_var])] += sample.amplitude
    return proj_statevector



state_result = get_projected_state_vector(
    results, "state", {"exp_block": 0.0, "ham_block": 0.0}
)
# print(state_result)

# # expected state after evolution
expected_state = (1 / normalization_exp) * scipy.linalg.expm(1j * Ham * EVOLUTION_TIME) @ init_state
relative_phase = np.angle(expected_state[0] / state_result[0])
state_result = state_result * np.exp(
    1j * relative_phase
)  # rotate according to a global phase



print("expected state:", np.transpose(expected_state))
print("resulting state:", state_result)
# assert np.linalg.norm(state_result - np.transpose(expected_state)) < EPS


expected state: [[0.98345056+0.j         0.29889057+0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.46088622j
  0.        +0.j         0.        +0.j        ]]
resulting state: [ 3.26954849e-01+1.06162250e-18j  2.31191991e-01-1.32377595e-15j
 -3.65156821e-16+8.21793999e-17j -2.93835404e-16+1.83857157e-16j
 -4.70615213e-15+2.72310552e-01j -4.49519854e-16-2.80861390e-01j
  1.10271636e-16-7.45842360e-18j  2.18157783e-16+4.61328698e-16j]


In [46]:

def normalize_final_state2(state_amplitudes):
    """
        Definition:
            Normalize the final state
        Args:
            state_amplitude_sums (dict): Amplitude and phase of each output qnum
            parsed_state_vector (dict): Results of the state vector simulation
        Output:
            final_state (nd_array 2*N^2x1): Normalized final state taking phases into account
    """

    values = np.array([state_amplitudes[i] for i in range(len(state_amplitudes))])
    global_phase = np.angle(values)[0]
    values = values / np.exp(1j * global_phase)

    normalization = np.linalg.norm(
            [state_amplitudes
            ]
        )

    final_state = values / normalization

    return final_state

In [47]:
# Normalize the final state
normalized_final_state = normalize_final_state2(state_result)
normalized_final_state

# Simplify the final state by neglecting small terms
simplified_final_state = simplify_final_state(normalized_final_state)
simplified_final_state

# Correct the normalization factor that comes from the normalization of initial state and transform the row vector to column vector
final_state = norm * simplified_final_state[..., None]
print("The final quantum state from quantum evolution", final_state)



The final quantum state from quantum evolution [[1.16809569+0.j        ]
 [0.82596838+0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.97287067j]
 [0.        -1.00341983j]
 [0.        +0.j        ]
 [0.        +0.j        ]]


In [48]:

# Get the final position and velocity vectors
final_results = post_process_final_state(final_state, B_padded, N, y_0)
y_final = final_results["y_final"]
ydot_final = final_results["ydot_final"]

print("Final Positions", y_final)
print("Final Velocities", ydot_final)

Final Positions [[0.68168389+0.j]
 [0.66894655+0.j]]
Final Velocities [[1.16809569+0.j]
 [0.82596838+0.j]]


In [49]:
# transform to original coordinates

x_final_results = back_coordinate_transformation(M, y_final, ydot_final)
x_final = x_final_results["x_final"]
xdot_final = x_final_results["xdot_final"]

print("Final Positions", x_final)
print("Final Velocities", xdot_final)


Final Positions [[0.4820233 +0.j]
 [0.66894655+0.j]]
Final Velocities [[0.82596838+0.j]
 [0.82596838+0.j]]


### **QSVT approach for Hamiltonian simulation:**


The QSVT is a general technique for applying block encoding of matrix polynomials, via quantum signal processing.  



In [50]:
EPS=0.1

import pyqsp
from pyqsp.angle_sequence import Polynomial, QuantumSignalProcessingPhases

pg_cos = pyqsp.poly.PolyCosineTX()
pcoefs_cos, scale_cos = pg_cos.generate(
    epsilon=EPS, tau=normalized_time, return_scale=True
)
poly_cos = Polynomial(pcoefs_cos)

pg_sin = pyqsp.poly.PolySineTX()
pcoefs_sin, scale_sin = pg_sin.generate(
    epsilon=EPS, tau=normalized_time, return_scale=True
)
poly_sin = Polynomial(pcoefs_sin)

2.589599151329187
R=1
[PolyCosineTX] rescaling by 0.5.
2.589599151329187
R=1
[PolySineTX] rescaling by 0.5.


In [51]:
ang_seq_cos = QuantumSignalProcessingPhases(
    poly_cos,
    signal_operator="Wx",
    method="laurent",
    measurement="x",
    tolerance=0.00001,  # relaxing the tolerance to get convergence
)

ang_seq_sin = QuantumSignalProcessingPhases(
    poly_sin,
    signal_operator="Wx",
    method="laurent",
    measurement="x",
    tolerance=0.00001,  # relaxing the tolerance to get convergence
)


# change W(x) to R(x), as the phases are in the W(x) conventions see Eq.()
def convert_phases_to_Rx_convention(ang_seq):

    phases = np.array(ang_seq)
    phases[1:-1] = phases[1:-1] - np.pi / 2
    phases[0] = phases[0] - np.pi / 4
    phases[-1] = phases[-1] + (2 * (len(phases) - 1) - 1) * np.pi / 4

    # verify conventions. minus is due to exp(-i*phi*z) in qsvt in comparison to qsp
    phases = -2 * phases

    return phases.tolist()

phases_cos = convert_phases_to_Rx_convention(ang_seq_cos)
phases_sin = convert_phases_to_Rx_convention(ang_seq_sin)

In [52]:
@qfunc
def identify_block(num_data_qubits: CInt, qvar: QArray[QBit], qbit: QBit):
    block_qubits = QNum("block_qubits", qvar.len - num_data_qubits, False, 0)
    data = QArray("data", length=num_data_qubits)

    bind(qvar, [block_qubits, data])
    qbit ^= block_qubits == 0
    bind([block_qubits, data], qvar)


# defining a qsvt for the specific example
@qfunc
def my_qsvt(phases: CArray[CReal], qsvt_aux: QBit, block_ham: QArray, data: QArray):
    combined_vars = QArray("combined_vars")
    bind([block_ham, data], combined_vars)

    qsvt(
        phase_seq=phases,
        proj_cnot_1=lambda qvar, qbit: identify_block(data.len, qvar, qbit),
        proj_cnot_2=lambda qvar, qbit: identify_block(data.len, qvar, qbit),
        u=lambda x: lcu_paulis(
            classiq_pauli_list,
            prepare_probs_ham,
            x[0 : block_ham.len],
            x[block_ham.len : x.len],
        ),
        qvar=combined_vars,
        aux=qsvt_aux,
    )

    bind(combined_vars, [block_ham, data])

In [53]:

@qfunc
def init_state_phase(state: QNum):
    """
        Definition:
            Imply -pi/2 phase to the last half (msb qubit) of the QNum state in order to add -i factor
        Args:
            state (QNum): Initial state with bare amplitudes
        Outputs:
            state (QNum): Initial state with phase
    """

    state_in_qubit = QArray("state_in_qubit")
    msb = QArray("msb", QBit)
    
    size = np.log2(init_state.size)
    allocate(size, msb)

    bind(state, state_in_qubit)
    repeat(state_in_qubit.len, lambda i: CX(state_in_qubit[i], msb[i]))
    control(msb[size-1], lambda: PHASE(pi/2, state_in_qubit[size-1]))
    bind(state_in_qubit, state)



@qfunc
def main(
    qsvt_aux: Output[QBit],
    block_ham: Output[QNum],
    state: Output[QNum],
    block_exp: Output[QBit],
):
    allocate(1, qsvt_aux)
    allocate(1, block_exp)
    allocate(lcu_size_ham, block_ham)
    prepare_amplitudes(amplitudes=list(normalized_init_state), out=state, bound=0.01)
    init_state_phase(state)
    within_apply(
        lambda: H(block_exp),
        lambda: (
            control(
                block_exp == 0,  # cosine
                lambda: my_qsvt(phases_cos, qsvt_aux, block_ham, state),
            ),
            control(
                block_exp == 1,  # sine
                lambda: (
                    U(0, 0, 0, np.pi / 2, qsvt_aux),  # for the i factor
                    my_qsvt(phases_sin, qsvt_aux, block_ham, state),
                ),
            ),
        ),
    )

qmod = create_model(main, execution_preferences=Execution_Prefs)

In [54]:
qprog = synthesize(qmod)
results = execute(qprog).result()
# results

In [55]:
state_result = get_projected_state_vector(
    results, "state", {"block_ham": 0.0, "qsvt_aux": 0.0, "block_exp": 0.0}
)
state_result

array([-1.42330265e-01-2.83112500e-02j, -1.00642696e-01-2.00190768e-02j,
       -4.31669283e-17-1.69778213e-17j,  1.12634417e-16-3.30394660e-16j,
        2.42854891e-02-1.22091398e-01j,  2.43184603e-02-1.22257156e-01j,
        1.47278202e-16+5.04945665e-16j,  6.61604234e-17+2.88695422e-16j])

In [56]:
expected_state = (
    1 / normalization_exp * scipy.linalg.expm(1j * Ham * EVOLUTION_TIME) @ init_state
)
relative_phase = np.angle(expected_state[0] / state_result[0])
state_result = state_result * np.exp(
    1j * relative_phase
)
print("expected state:", np.transpose(expected_state))
print("resulting state:", state_result)
# assert np.linalg.norm(state_result - np.transpose(expected_state)) < EPS

expected state: [[0.98345056+0.j         0.29889057+0.j         0.        +0.j
  0.        +0.j         0.        +0.j         0.        +0.46088622j
  0.        +0.j         0.        +0.j        ]]
resulting state: [ 1.45118680e-01-7.24515621e-19j  1.02614402e-01+2.59375006e-15j
  4.56496965e-17+8.23014729e-18j -4.60133779e-17+3.46020104e-16j
 -6.13706112e-16+1.24483310e-01j  1.99261628e-16+1.24652315e-01j
 -2.42958305e-16-4.66510724e-16j -1.21210852e-16-2.70240962e-16j]


In [57]:
# Normalize the final state
normalized_final_state = normalize_final_state2(state_result)
normalized_final_state

# Simplify the final state by neglecting small terms
simplified_final_state = simplify_final_state(normalized_final_state)
simplified_final_state

# Correct the normalization factor that comes from the normalization of initial state and transform the row vector to column vector
final_state = norm * simplified_final_state[..., None]
print("The final quantum state from quantum evolution", final_state)



The final quantum state from quantum evolution [[1.15980466+0.j        ]
 [0.82010574+0.j        ]
 [0.        +0.j        ]
 [0.        +0.j        ]
 [0.        +0.99488449j]
 [0.        +0.9962352j ]
 [0.        +0.j        ]
 [0.        +0.j        ]]


In [58]:
# Get the final position and velocity vectors
final_results = post_process_final_state(final_state, B_padded, N, y_0)
y_final = final_results["y_final"]
ydot_final = final_results["ydot_final"]

print("Final Positions", y_final)
print("Final Velocities", ydot_final)

Final Positions [[ 1.62433031+0.j]
 [-0.6641568 +0.j]]
Final Velocities [[1.15980466+0.j]
 [0.82010574+0.j]]


In [59]:
# transform to original coordinates

x_final_results = back_coordinate_transformation(M, y_final, ydot_final)
x_final = x_final_results["x_final"]
xdot_final = x_final_results["xdot_final"]

print("Final Positions", x_final)
print("Final Velocities", xdot_final)

Final Positions [[ 1.14857498+0.j]
 [-0.6641568 +0.j]]
Final Velocities [[0.82010574+0.j]
 [0.82010574+0.j]]
